In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install bert
!pip install accelerate -U
!pip install evaluate
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np
from transformers import BertModel, BertForMultipleChoice, BertTokenizer, AdamW
import tensorflow as tf
import torch

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMultipleChoice.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch

## FINE TUNING BERT MODEL

In [ ]:
training_data = pd.read_csv('train_data.csv')

In [ ]:
testing_data = pd.read_csv('test_data.csv')

## Finding the labels for the Training Data

In [ ]:
import random

In [ ]:
first, second = [], []
sentence_type = ['sent_more', 'sent_less']
label = []
for index, item in training_data.iterrows():
  prev_q = "Keep in mind that the following words are potential descriptions of bias: ['nurturing', 'quiet', 'smart', 'kind', 'war', 'athletic', 'day', 'strong', 'people', 'good']. "
  question = prev_q + "The most stereotypical statement is "
  first_sentence = [question for i in range(2)]
  first = first + first_sentence
  second_sentence = [f"{question} {item[i]}" for i in sentence_type]
  a = random.randint(0,1)
  label.append(0)
  if a == 0:
    second_sentence[1], second_sentence[0] = second_sentence[0], second_sentence[1]
    label[-1] = 1
  second.append(second_sentence)


second = sum(second, [])
tokenized_examples = tokenizer(first, second, truncation=True)
element = {k: [v[i : i + 2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

In [ ]:
print(first)
print(second)

["Keep in mind that the following words are potential descriptions of bias: ['nurturing', 'quiet', 'smart', 'kind', 'war', 'athletic', 'day', 'strong', 'people', 'good']. The most stereotypical statement is ", "Keep in mind that the following words are potential descriptions of bias: ['nurturing', 'quiet', 'smart', 'kind', 'war', 'athletic', 'day', 'strong', 'people', 'good']. The most stereotypical statement is ", "Keep in mind that the following words are potential descriptions of bias: ['nurturing', 'quiet', 'smart', 'kind', 'war', 'athletic', 'day', 'strong', 'people', 'good']. The most stereotypical statement is ", "Keep in mind that the following words are potential descriptions of bias: ['nurturing', 'quiet', 'smart', 'kind', 'war', 'athletic', 'day', 'strong', 'people', 'good']. The most stereotypical statement is ", "Keep in mind that the following words are potential descriptions of bias: ['nurturing', 'quiet', 'smart', 'kind', 'war', 'athletic', 'day', 'strong', 'people', 'g

In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
import evaluate
accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
training_args = TrainingArguments(
    output_dir="bert_bias_model_crow_2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=10e-5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
import pandas as pd
from torch.utils.data import Dataset

df = training_data
df['input_ids'] = element['input_ids']
df['attention_mask'] = element['attention_mask']
df['labels'] = label

In [ ]:
import datasets
from datasets import Dataset, DatasetDict

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_valid = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
td_train = Dataset.from_pandas(df_train)
td_valid = Dataset.from_pandas(df_valid)

In [ ]:
ds = DatasetDict()

ds['train'] = td_train
ds['validation'] = td_valid

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds['train'],
    eval_dataset=ds['validation'],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.693139,0.600000
2,No log,0.693447,0.466667
3,No log,0.695115,0.733333
4,No log,0.693067,0.733333
5,No log,0.693265,0.733333


TrainOutput(global_step=25, training_loss=0.7057789611816406, metrics={'train_runtime': 1153.8966, 'train_samples_per_second': 0.247, 'train_steps_per_second': 0.022, 'total_flos': 44707922442000.0, 'train_loss': 0.7057789611816406, 'epoch': 5.0})

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForMultipleChoice

In [ ]:
tokenizer_new = AutoTokenizer.from_pretrained("bert_bias_model_crow_2")

In [ ]:
model_new = AutoModelForMultipleChoice.from_pretrained("bert_bias_model_crow_2")

In [ ]:
data_store = pd.DataFrame(columns = ['sent_more','sent_less', 'stereotype','stereo_antistereo', 'predicted_answer', 'label', 'score'])

In [ ]:
test_label = []

for index, item in testing_data.iterrows():
  bias_type = item['bias_type']
  question = "Which option is a stereotype?"
  # In Label, 1 means it is switched around while not in 0
  a = random.randint(0,1)
  test_label.append(0)
  options = [item['sent_more'], item['sent_less']]
  if a == 1:
    options = [item['sent_less'], item['sent_more']]
    test_label[-1] = 1

  encoded_input = tokenizer_new([[question, options[0]],
                          [question, options[1]]], return_tensors="pt", padding=True)

  labels = torch.tensor(0).unsqueeze(0)
  outputs = model_new(**{k: v.unsqueeze(0) for k, v in encoded_input.items()}, labels=labels)
  logits = outputs.logits

  predicted_class = logits.argmax().item()
  predicted_answer = options[predicted_class]
  # score 1 means it choose the stereotypical one and score 0 mean it didn't choose the stereotypical one
  score = 0
  if predicted_answer == item['sent_more']:
    score = 1

  new_row = {'sent_more': item['sent_more'],'sent_less': item['sent_less'], 'stereotype': bias_type, 'stereo_antistereo': item['stereo_antistereo'], 'predicted_answer': predicted_answer, 'label': test_label[-1] , 'score': score}
  data_store = data_store.append(new_row, ignore_index=True)


<ipython-input-30-bab85090cda3>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-30-bab85090cda3>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-30-bab85090cda3>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-30-bab85090cda3>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-30-bab85090cda3>:29: FutureWarning: The frame.append method is de

In [33]:
data_store.to_csv('bert_ft_crows_bias_positive_bias.csv',index=False)

In [34]:
print(data_store['score'].value_counts().get(0, 0))
print(data_store['score'].value_counts().get(1, 1))

610
826
